In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Function to preprocess text
def preprocess_text(text):
    # Implement your preprocessing steps here (e.g., remove punctuation, stopwords, tokenize, lemmatize)
    return text

# Path to the "Dataset" folder in your Google Drive
folder_path = '/content/drive/My Drive/Dataset'

# List all CSV files in the "Dataset" folder
csv_files = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path) if file_name.endswith('.csv')]

results = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    question = df.iloc[0, 0]  # Assuming the question is in the first column
    model_answer = preprocess_text(df.iloc[0, -1])  # Assuming the model answer is in the last column

    # Preprocess sample answers
    sample_answers = [preprocess_text(df.iloc[0, i]) for i in range(1, 5)]  # Assuming sample answers are in columns 2 to 5

    # Word2Vec for Cosine Similarity
    # Train Word2Vec model
    text_data = [preprocess_text(df.iloc[0, i]) for i in range(1, 6)]
    tokenized_data = [text.split() for text in text_data]
    word2vec_model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

    # Calculate cosine similarity between each sample answer and model answer using Word2Vec
    similarity_scores_word2vec = []
    for sample_answer in sample_answers:
        sample_answer_vec = [word2vec_model.wv[word] for word in sample_answer.split() if word in word2vec_model.wv]
        model_answer_vec = [word2vec_model.wv[word] for word in model_answer.split() if word in word2vec_model.wv]
        if len(sample_answer_vec) > 0 and len(model_answer_vec) > 0:
            similarity_score = cosine_similarity(sample_answer_vec, model_answer_vec).mean()
            similarity_scores_word2vec.append(similarity_score)
        else:
            similarity_scores_word2vec.append(0.0)

    # SBERT for Similarity
    sbert_model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

    # Encode model answer
    model_answer_embedding = sbert_model.encode(model_answer)

    # Calculate cosine similarity between each sample answer and model answer using SBERT
    similarity_scores_sbert = []
    for sample_answer in sample_answers:
        sample_answer_embedding = sbert_model.encode(sample_answer)
        similarity_score = cosine_similarity([model_answer_embedding], [sample_answer_embedding])[0][0]
        similarity_scores_sbert.append(similarity_score)

    # Compare Cosine Similarity vs. SBERT
    avg_similarity_word2vec = sum(similarity_scores_word2vec) / len(similarity_scores_word2vec)
    avg_similarity_sbert = sum(similarity_scores_sbert) / len(similarity_scores_sbert)

    results.append({
        'Question': question,
        'Model Answer': model_answer,
        'Sample Answers': sample_answers,
        'Similarity (Word2Vec)': avg_similarity_word2vec,
        'Similarity (SBERT)': avg_similarity_sbert
    })

# Display results
results_df = pd.DataFrame(results)
results_df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

,Question,Model Answer,Sample Answers,Similarity (Word2Vec),Similarity (SBERT)
0,1.0,The decision tree Algorithm belongs to the fam...,"[Decision Tree Classification Algorithm, Decis...",0.022259,0.781217


In [ ]:
from google.colab import drive
import os
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Mount Google Drive
drive.mount('/content/drive')

# Path to the "Dataset" folder in your Google Drive
folder_path = '/content/drive/My Drive/Dataset'

# List all CSV files in the "Dataset" folder
csv_files = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path) if file_name.endswith('.csv')]

results = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file, skiprows=1)  # Skip the header row
    if len(df.columns) < 7:
        continue  # Skip this file if it doesn't have enough columns

    question = df.iloc[0, 1]  # Assuming the question is in the second column
    model_answers = df.iloc[:, -1].tolist()  # Assuming the model answers are in the last column
    sample_answers = [df.iloc[:, i].tolist() for i in range(2, 7)]  # Assuming sample answers are in columns 3 to 7

    # Word2Vec for Cosine Similarity
    # Train Word2Vec model
    text_data = [df.iloc[:, i].tolist() for i in range(2, 7)]
    tokenized_data = [[str(text) for text in column_data] for column_data in text_data]
    word2vec_model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

    # Calculate cosine similarity between each sample answer and model answer using Word2Vec
    similarity_scores_word2vec = []
    for model_answer, sample_answer_list in zip(model_answers, sample_answers):
        model_answer_vec = [word2vec_model.wv[word] for word in str(model_answer).split() if word in word2vec_model.wv]
        similarity_scores = []
        for sample_answer in sample_answer_list:
            sample_answer_vec = [word2vec_model.wv[word] for word in str(sample_answer).split() if word in word2vec_model.wv]
            if len(sample_answer_vec) > 0 and len(model_answer_vec) > 0:
                similarity_score = cosine_similarity(sample_answer_vec, model_answer_vec).mean()
                similarity_scores.append(similarity_score)
            else:
                similarity_scores.append(0.0)
        similarity_scores_word2vec.append(similarity_scores)

    # SBERT for Similarity
    sbert_model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

    # Encode model answers
    model_answers_embeddings = [sbert_model.encode(str(answer)) for answer in model_answers]

    # Calculate cosine similarity between each sample answer and model answer using SBERT
    similarity_scores_sbert = []
    for model_answer_embedding, sample_answer_list in zip(model_answers_embeddings, sample_answers):
        similarity_scores = []
        for sample_answer in sample_answer_list:
            sample_answer_embedding = sbert_model.encode(str(sample_answer))
            similarity_score = cosine_similarity([model_answer_embedding], [sample_answer_embedding])[0][0]
            similarity_scores.append(similarity_score)
        similarity_scores_sbert.append(similarity_scores)

    # Compare Cosine Similarity vs. SBERT
    avg_similarity_word2vec = []
    for question_scores in similarity_scores_word2vec:
        if isinstance(question_scores, float):
            avg_similarity_word2vec.append(question_scores)
        else:
            avg_similarity_word2vec.append(sum(question_scores) / len(question_scores))

    avg_similarity_sbert = []
    for question_scores in similarity_scores_sbert:
        if isinstance(question_scores, float):
            avg_similarity_sbert.append(question_scores)
        else:
            avg_similarity_sbert.append(sum(question_scores) / len(question_scores))

    results.append({
        'Question': question,
        'Model Answers': model_answers,
        'Sample Answers': sample_answers,
        'Similarity (Word2Vec)': avg_similarity_word2vec,
        'Similarity (SBERT)': avg_similarity_sbert
    })

# Display results
results_df = pd.DataFrame(results)
results_df


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Question,Model Answers,Sample Answers,Similarity (Word2Vec),Similarity (SBERT)
0,What is Machine Learning?,[Machine learning is a subfield of artificial ...,[[Machine learning is a subfield of artificial...,"[0.0, 0.0, 0.0, 0.0, 0.5000000596046448]","[0.2657739007845521, 0.3741172756999731, 0.325..."


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Load CSV file
df = pd.read_csv('/content/drive/My Drive/Dataset/ml_dataset.csv', skiprows=1)

# Process only the first 5 questions
results = []
question_count = 0

for index, row in df.iterrows():
    if question_count >= 5:
        break  # Exit the loop if we have processed 5 questions

    question = row[1]  # Assuming the question is in the second column
    model_answer = row[6]  # Assuming the model answer is in the last column
    sample_answers = row[2:6].tolist()  # Assuming sample answers are in columns 3 to 6

    # Word2Vec for Cosine Similarity
    # Train Word2Vec model
    text_data = [model_answer] + sample_answers
    tokenized_data = [str(text).split() for text in text_data]
    word2vec_model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

    # Calculate cosine similarity between each sample answer and model answer using Word2Vec
    similarity_scores_word2vec = []
    for sample_answer in sample_answers:
        model_answer_vec = [word2vec_model.wv[word] for word in str(model_answer).split() if word in word2vec_model.wv]
        sample_answer_vec = [word2vec_model.wv[word] for word in str(sample_answer).split() if word in word2vec_model.wv]
        if len(sample_answer_vec) > 0 and len(model_answer_vec) > 0:
            similarity_score = cosine_similarity(sample_answer_vec, model_answer_vec).mean()
        else:
            similarity_score = 0.0
        similarity_scores_word2vec.append(similarity_score)

    # SBERT for Similarity
    sbert_model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

    # Encode model answer
    model_answer_embedding = sbert_model.encode(str(model_answer))

    # Calculate cosine similarity between each sample answer and model answer using SBERT
    similarity_scores_sbert = []
    for sample_answer in sample_answers:
        sample_answer_embedding = sbert_model.encode(str(sample_answer))
        similarity_score = cosine_similarity([model_answer_embedding], [sample_answer_embedding])[0][0]
        similarity_scores_sbert.append(similarity_score)

    # Append results for this question to the list
    results.append({
        'Question': question,
        'Model Answer': model_answer,
        'Sample Answers': sample_answers,
        'Similarity (Word2Vec)': similarity_scores_word2vec,
        'Similarity (SBERT)': similarity_scores_sbert
    })

    question_count += 1

# Display results
results_df = pd.DataFrame(results)
results_df

,Question,Model Answer,Sample Answers,Similarity (Word2Vec),Similarity (SBERT)
0,Decision Tree Classification Algorithm,A decision tree is a non-parametric supervised...,[Decision Tree is a Supervised learning techni...,"[0.027359316, 0.021366088, 0.027940927, 0.0275...","[0.77213585, 0.86438465, 0.875097, 0.88478905]"
1,What is Machine Learning?,Machine learning (ML) is a branch of artificia...,[Machine learning is a subfield of artificial ...,"[0.02165151, 0.016202396, 0.013175168, 0.01598...","[0.6981575, 0.73919064, 0.38095444, 0.7365947]"
2,What is overfitting in machine learning?,Overfitting refers to a model that models the ...,[A statistical model is said to be overfitted ...,"[0.033889513, 0.033489507, 0.030074129, 0.0301...","[0.8601811, 0.7698139, 0.82044244, 0.8054892]"
3,"What is regularization, and why is it importan...",Regularization means restricting a model to av...,[While developing machine learning models you ...,"[0.020813834, 0.024386007, 0.01945477, 0.01770...","[0.64998215, 0.754961, 0.7667507, 0.8127868]"
4,"What is cross-validation, and why is it import...",Cross validation is a technique for assessing ...,[Cross validation is a technique used in machi...,"[0.031890023, 0.03366408, 0.028353577, 0.03753...","[0.7595998, 0.7571845, 0.5400534, 0.808226]"


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.metrics import jaccard_score

# Load CSV file
df = pd.read_csv('/content/drive/My Drive/Dataset/ml_dataset.csv', skiprows=0)

# Process only the first 5 questions
results = []
question_count = 0

for index, row in df.iterrows():
    if question_count >= 5:
        break  # Exit the loop if we have processed 5 questions

    question = row[1]  # Assuming the question is in the second column
    model_answer = row[6]  # Assuming the model answer is in the last column
    sample_answers = row[2:6].tolist()  # Assuming sample answers are in columns 3 to 6

    # Tokenize the model answer and sample answers
    model_answer_tokens = set(str(model_answer).split())
    sample_answer_tokens = [set(str(sample_answer).split()) for sample_answer in sample_answers]

    # Calculate Jaccard similarity between model answer and each sample answer
    similarity_scores_jaccard = [jaccard_score(model_answer_tokens, sample_answer_tokens[i]) for i in range(len(sample_answers))]

    # Append results for this question to the list
    results.append({
        'Question': question,
        'Model Answer': model_answer,
        'Sample Answers': sample_answers,
        'Similarity (Jaccard)': similarity_scores_jaccard
    })

    question_count += 1

# Display results
results_df = pd.DataFrame(results)
results_df

ValueError: Found input variables with inconsistent numbers of samples: [67, 111]

In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Load CSV file
df = pd.read_csv('/content/drive/My Drive/Dataset/ml_dataset.csv', skiprows=0)

# Process only the first 5 questions
results = []
question_count = 0

for index, row in df.iterrows():
    if question_count >= 5:
        break  # Exit the loop if we have processed 5 questions

    question = row[1]  # Assuming the question is in the second column
    model_answer = row[6]  # Assuming the model answer is in the last column
    sample_answers = row[2:6].tolist()  # Assuming sample answers are in columns 3 to 6

    # Word2Vec for Cosine Similarity
    # Train Word2Vec model
    text_data = [model_answer] + sample_answers
    tokenized_data = [str(text).split() for text in text_data]
    word2vec_model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

    # Calculate cosine similarity between each sample answer and model answer using Word2Vec
    similarity_scores_word2vec = []
    for sample_answer in sample_answers:
        model_answer_vec = [word2vec_model.wv[word] for word in str(model_answer).split() if word in word2vec_model.wv]
        sample_answer_vec = [word2vec_model.wv[word] for word in str(sample_answer).split() if word in word2vec_model.wv]
        if len(sample_answer_vec) > 0 and len(model_answer_vec) > 0:
            similarity_score = cosine_similarity(sample_answer_vec, model_answer_vec).mean()
        else:
            similarity_score = 0.0
        similarity_scores_word2vec.append(similarity_score)

    # SBERT for Similarity
    sbert_model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

    # Encode model answer
    model_answer_embedding = sbert_model.encode(str(model_answer))

    # Calculate cosine similarity between each sample answer and model answer using SBERT
    similarity_scores_sbert = []
    for sample_answer in sample_answers:
        sample_answer_embedding = sbert_model.encode(str(sample_answer))
        similarity_score = cosine_similarity([model_answer_embedding], [sample_answer_embedding])[0][0]
        similarity_scores_sbert.append(similarity_score)

    # Append results for this question to the list
    results.append({
        'Question': question,
        'Model Answer': model_answer,
        'Sample Answers': sample_answers,
        'Similarity (Word2Vec)': similarity_scores_word2vec,
        'Similarity (SBERT)': similarity_scores_sbert
    })

    question_count += 1

# Display results
results_df = pd.DataFrame(results)
results_df

,Question,Model Answer,Sample Answers,Similarity (Word2Vec),Similarity (SBERT)
0,Decision Tree Classification Algorithm,A decision tree is a non-parametric supervised...,[Decision Tree is a Supervised learning techni...,"[0.027359316, 0.021366088, 0.027940927, 0.0275...","[0.77213585, 0.86438465, 0.875097, 0.88478905]"
1,What is Machine Learning?,Machine learning (ML) is a branch of artificia...,[Machine learning is a subfield of artificial ...,"[0.02165151, 0.016202396, 0.013175168, 0.01598...","[0.6981575, 0.73919064, 0.38095444, 0.7365947]"
2,What is overfitting in machine learning?,Overfitting refers to a model that models the ...,[A statistical model is said to be overfitted ...,"[0.033889513, 0.033489507, 0.030074129, 0.0301...","[0.8601811, 0.7698139, 0.82044244, 0.8054892]"
3,"What is regularization, and why is it importan...",Regularization means restricting a model to av...,[While developing machine learning models you ...,"[0.020813834, 0.024386007, 0.01945477, 0.01770...","[0.64998215, 0.754961, 0.7667507, 0.8127868]"
4,"What is cross-validation, and why is it import...",Cross validation is a technique for assessing ...,[Cross validation is a technique used in machi...,"[0.031890023, 0.03366408, 0.028353577, 0.03753...","[0.7595998, 0.7571845, 0.5400534, 0.808226]"


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.metrics import jaccard_score

# Load CSV file
df = pd.read_csv('/content/drive/My Drive/Dataset/ml_dataset.csv', skiprows=0)

# Process only the first 5 questions
results = []
question_count = 0

for index, row in df.iterrows():
    if question_count >= 5:
        break  # Exit the loop if we have processed 5 questions

    question = row[1]  # Assuming the question is in the second column
    model_answer = row[6]  # Assuming the model answer is in the last column
    sample_answers = row[2:6].tolist()  # Assuming sample answers are in columns 3 to 6

    # Word2Vec for Cosine Similarity
    # Train Word2Vec model
    text_data = [model_answer] + sample_answers
    tokenized_data = [str(text).split() for text in text_data]
    word2vec_model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

    # Calculate cosine similarity between each sample answer and model answer using Word2Vec
    similarity_scores_word2vec = []
    for sample_answer in sample_answers:
        model_answer_vec = [word2vec_model.wv[word] for word in str(model_answer).split() if word in word2vec_model.wv]
        sample_answer_vec = [word2vec_model.wv[word] for word in str(sample_answer).split() if word in word2vec_model.wv]
        if len(sample_answer_vec) > 0 and len(model_answer_vec) > 0:
            similarity_score = cosine_similarity(sample_answer_vec, model_answer_vec).mean()
        else:
            similarity_score = 0.0
        similarity_scores_word2vec.append(similarity_score)

    # SBERT for Similarity
    sbert_model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

    # Encode model answer
    model_answer_embedding = sbert_model.encode(str(model_answer))

    # Calculate cosine similarity between each sample answer and model answer using SBERT
    similarity_scores_sbert = []
    for sample_answer in sample_answers:
        sample_answer_embedding = sbert_model.encode(str(sample_answer))
        similarity_score = cosine_similarity([model_answer_embedding], [sample_answer_embedding])[0][0]
        similarity_scores_sbert.append(similarity_score)

    # Calculate Jaccard similarity between model answer and each sample answer
    model_answer_tokens = set(str(model_answer).split())
    # Calculate Jaccard similarity between model answer and each sample answer
    # Calculate Jaccard similarity between model answer and each sample answer
# Calculate Jaccard similarity between model answer and each sample answer
similarity_scores_jaccard = []
for i in range(len(sample_answers)):
    sample_answer = str(sample_answers[i])
    if sample_answer.lower() == 'nan':
        similarity_scores_jaccard.append(0.0)  # Skip 'nan' values
        continue
    sample_answer_tokens = set(sample_answer.split())
    similarity_score = jaccard_score(set(model_answer_tokens), sample_answer_tokens)
    similarity_scores_jaccard.append(similarity_score)



    # Append results for this question to the list
    results.append({
        'Question': question,
        'Model Answer': model_answer,
        'Sample Answers': sample_answers,
        'Similarity (Word2Vec)': similarity_scores_word2vec,
        'Similarity (SBERT)': similarity_scores_sbert,
        'Similarity (Jaccard)': similarity_scores_jaccard
    })

    question_count += 1

# Display results
results_df = pd.DataFrame(results)
results_df

""


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from transformers import BertTokenizer, BertModel
import torch

# Load CSV file
df = pd.read_csv('/content/drive/My Drive/Dataset/ml_dataset.csv', skiprows=0)

# Process only the first 5 questions
results = []
question_count = 0

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

for index, row in df.iterrows():
    if question_count >= 5:
        break  # Exit the loop if we have processed 5 questions

    question = row[1]  # Assuming the question is in the second column
    model_answer = row[6]  # Assuming the model answer is in the last column
    sample_answers = row[2:6].tolist()  # Assuming sample answers are in columns 3 to 6

    # Word2Vec for Cosine Similarity
    # Train Word2Vec model
    text_data = [model_answer] + sample_answers
    tokenized_data = [str(text).split() for text in text_data]
    word2vec_model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

    # Calculate cosine similarity between each sample answer and model answer using Word2Vec
    similarity_scores_word2vec = []
    for sample_answer in sample_answers:
        model_answer_vec = [word2vec_model.wv[word] for word in str(model_answer).split() if word in word2vec_model.wv]
        sample_answer_vec = [word2vec_model.wv[word] for word in str(sample_answer).split() if word in word2vec_model.wv]
        if len(sample_answer_vec) > 0 and len(model_answer_vec) > 0:
            similarity_score = cosine_similarity(sample_answer_vec, model_answer_vec).mean()
        else:
            similarity_score = 0.0
        similarity_scores_word2vec.append(similarity_score)

    # BERT embeddings
    # Tokenize model answer and sample answers
    model_answer_tokens = tokenizer.tokenize(model_answer)
    sample_answers_tokens = [tokenizer.tokenize(sample_answer) for sample_answer in sample_answers]

    # Convert tokens to IDs
    model_answer_ids = tokenizer.convert_tokens_to_ids(model_answer_tokens)
    sample_answers_ids = [tokenizer.convert_tokens_to_ids(sample_answer_tokens) for sample_answer_tokens in sample_answers_tokens]

    # Convert IDs to tensors
    model_answer_tensor = torch.tensor([model_answer_ids])
    sample_answers_tensors = [torch.tensor([sample_answer_ids]) for sample_answer_ids in sample_answers_ids]

    # Get BERT embeddings for model answer
    with torch.no_grad():
        model_answer_embedding = bert_model(model_answer_tensor)[0][:, 0, :].numpy()

    # Calculate cosine similarity between each sample answer and model answer using BERT embeddings
    similarity_scores_bert = []
    for sample_answer_tensor in sample_answers_tensors:
        with torch.no_grad():
            sample_answer_embedding = bert_model(sample_answer_tensor)[0][:, 0, :].numpy()
        similarity_score = cosine_similarity(sample_answer_embedding, model_answer_embedding).mean()
        similarity_scores_bert.append(similarity_score)

    # SBERT for Similarity
    sbert_model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

    # Encode model answer
    model_answer_embedding_sbert = sbert_model.encode(str(model_answer))

    # Calculate cosine similarity between each sample answer and model answer using SBERT
    similarity_scores_sbert = []
    for sample_answer in sample_answers:
        sample_answer_embedding_sbert = sbert_model.encode(str(sample_answer))
        similarity_score = cosine_similarity([model_answer_embedding_sbert], [sample_answer_embedding_sbert])[0][0]
        similarity_scores_sbert.append(similarity_score)

    # Append results for this question to the list
    results.append({
        'Question': question,
        'Model Answer': model_answer,
        'Sample Answers': sample_answers,
        'Similarity (Word2Vec)': similarity_scores_word2vec,
        'Similarity (BERT)': similarity_scores_bert,
        'Similarity (SBERT)': similarity_scores_sbert
    })

    question_count += 1

# Display results
results_df = pd.DataFrame(results)
results_df

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

,Question,Model Answer,Sample Answers,Similarity (Word2Vec),Similarity (BERT),Similarity (SBERT)
0,Decision Tree Classification Algorithm,A decision tree is a non-parametric supervised...,[Decision Tree is a Supervised learning techni...,"[0.027359316, 0.021366088, 0.027940927, 0.0275...","[0.81093395, 0.8761315, 0.9076638, 0.8042929]","[0.77213585, 0.86438465, 0.875097, 0.88478905]"
1,What is Machine Learning?,Machine learning (ML) is a branch of artificia...,[Machine learning is a subfield of artificial ...,"[0.02165151, 0.016202396, 0.013175168, 0.01598...","[0.7180903, 0.7633122, 0.8259984, 0.7545961]","[0.6981575, 0.73919064, 0.38095444, 0.7365947]"
2,What is overfitting in machine learning?,Overfitting refers to a model that models the ...,[A statistical model is said to be overfitted ...,"[0.033889513, 0.033489507, 0.030074129, 0.0301...","[0.83725953, 0.56676894, 0.6560221, 0.7693534]","[0.8601811, 0.7698139, 0.82044244, 0.8054892]"
3,"What is regularization, and why is it importan...",Regularization means restricting a model to av...,[While developing machine learning models you ...,"[0.020813834, 0.024386007, 0.01945477, 0.01770...","[0.66867507, 0.79272395, 0.7334994, 0.7907195]","[0.64998215, 0.754961, 0.7667507, 0.8127868]"
4,"What is cross-validation, and why is it import...",Cross validation is a technique for assessing ...,[Cross validation is a technique used in machi...,"[0.031890023, 0.03366408, 0.028353577, 0.03753...","[0.8100376, 0.68056935, 0.650217, 0.5829565]","[0.7595998, 0.7571845, 0.5400534, 0.808226]"


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from transformers import BertTokenizer, BertModel
import torch

# Load CSV file
df = pd.read_csv('/content/drive/My Drive/Dataset/ml_dataset.csv', skiprows=0)

# Process only the first 5 questions
results = []
question_count = 0

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

for index, row in df.iterrows():
    if question_count >= 5:
        break  # Exit the loop if we have processed 5 questions

    question = row[1]  # Assuming the question is in the second column
    model_answer = row[6]  # Assuming the model answer is in the last column
    sample_answers = row[2:6].tolist()  # Assuming sample answers are in columns 3 to 6

    # Word2Vec for Cosine Similarity
    # Train Word2Vec model
    text_data = [model_answer] + sample_answers
    tokenized_data = [str(text).split() for text in text_data]
    word2vec_model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

    # Calculate cosine similarity between each sample answer and model answer using Word2Vec
    similarity_scores_word2vec = []
    for sample_answer in sample_answers:
        model_answer_vec = [word2vec_model.wv[word] for word in str(model_answer).split() if word in word2vec_model.wv]
        sample_answer_vec = [word2vec_model.wv[word] for word in str(sample_answer).split() if word in word2vec_model.wv]
        if len(sample_answer_vec) > 0 and len(model_answer_vec) > 0:
            similarity_score = cosine_similarity(sample_answer_vec, model_answer_vec).mean()
        else:
            similarity_score = 0.0
        similarity_scores_word2vec.append(similarity_score)

    # TF-IDF for Cosine Similarity
    # Calculate TF-IDF vectors
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([model_answer] + sample_answers)

    # Calculate cosine similarity between each sample answer and model answer using TF-IDF
    similarity_scores_tfidf = []
    for i in range(1, tfidf_matrix.shape[0]):
        similarity_score = cosine_similarity(tfidf_matrix[0], tfidf_matrix[i]).mean()
        similarity_scores_tfidf.append(similarity_score)

    # BERT embeddings
    # Tokenize model answer and sample answers
    model_answer_tokens = tokenizer.tokenize(model_answer)
    sample_answers_tokens = [tokenizer.tokenize(sample_answer) for sample_answer in sample_answers]

    # Convert tokens to IDs
    model_answer_ids = tokenizer.convert_tokens_to_ids(model_answer_tokens)
    sample_answers_ids = [tokenizer.convert_tokens_to_ids(sample_answer_tokens) for sample_answer_tokens in sample_answers_tokens]

    # Convert IDs to tensors
    model_answer_tensor = torch.tensor([model_answer_ids])
    sample_answers_tensors = [torch.tensor([sample_answer_ids]) for sample_answer_ids in sample_answers_ids]

    # Get BERT embeddings for model answer
    with torch.no_grad():
        model_answer_embedding = bert_model(model_answer_tensor)[0][:, 0, :].numpy()

    # Calculate cosine similarity between each sample answer and model answer using BERT embeddings
    similarity_scores_bert = []
    for sample_answer_tensor in sample_answers_tensors:
        with torch.no_grad():
            sample_answer_embedding = bert_model(sample_answer_tensor)[0][:, 0, :].numpy()
        similarity_score = cosine_similarity(sample_answer_embedding, model_answer_embedding).mean()
        similarity_scores_bert.append(similarity_score)

    # SBERT for Similarity
    sbert_model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

    # Encode model answer
    model_answer_embedding_sbert = sbert_model.encode(str(model_answer))

    # Calculate cosine similarity between each sample answer and model answer using SBERT
    similarity_scores_sbert = []
    for sample_answer in sample_answers:
        sample_answer_embedding_sbert = sbert_model.encode(str(sample_answer))
        similarity_score = cosine_similarity([model_answer_embedding_sbert], [sample_answer_embedding_sbert])[0][0]
        similarity_scores_sbert.append(similarity_score)

    # Append results for this question to the list
    results.append({
        'Question': question,
        'Model Answer': model_answer,
        'Sample Answers': sample_answers,
        'Similarity (Word2Vec)': similarity_scores_word2vec,
        'Similarity (TF-IDF)': similarity_scores_tfidf,
        'Similarity (BERT)': similarity_scores_bert,
        'Similarity (SBERT)': similarity_scores_sbert
    })

    question_count += 1

# Display results
results_df = pd.DataFrame(results)
results_df

,Question,Model Answer,Sample Answers,Similarity (Word2Vec),Similarity (TF-IDF),Similarity (BERT),Similarity (SBERT)
0,Decision Tree Classification Algorithm,A decision tree is a non-parametric supervised...,[Decision Tree is a Supervised learning techni...,"[0.027359316, 0.021366088, 0.027940927, 0.0275...","[0.6141617631446934, 0.469188688945496, 0.5958...","[0.81093395, 0.8761315, 0.9076638, 0.8042929]","[0.77213585, 0.86438465, 0.875097, 0.88478905]"
1,What is Machine Learning?,Machine learning (ML) is a branch of artificia...,[Machine learning is a subfield of artificial ...,"[0.02165151, 0.016202396, 0.013175168, 0.01598...","[0.3197806135632348, 0.3932176985464114, 0.325...","[0.7180903, 0.7633122, 0.8259984, 0.7545961]","[0.6981575, 0.73919064, 0.38095444, 0.7365947]"
2,What is overfitting in machine learning?,Overfitting refers to a model that models the ...,[A statistical model is said to be overfitted ...,"[0.033889513, 0.033489507, 0.030074129, 0.0301...","[0.5238203555006908, 0.5708784327257022, 0.447...","[0.83725953, 0.56676894, 0.6560221, 0.7693534]","[0.8601811, 0.7698139, 0.82044244, 0.8054892]"
3,"What is regularization, and why is it importan...",Regularization means restricting a model to av...,[While developing machine learning models you ...,"[0.020813834, 0.024386007, 0.01945477, 0.01770...","[0.24941959441809208, 0.36256187333858153, 0.2...","[0.66867507, 0.79272395, 0.7334994, 0.7907195]","[0.64998215, 0.754961, 0.7667507, 0.8127868]"
4,"What is cross-validation, and why is it import...",Cross validation is a technique for assessing ...,[Cross validation is a technique used in machi...,"[0.031890023, 0.03366408, 0.028353577, 0.03753...","[0.409154911536391, 0.4039990957034616, 0.3228...","[0.8100376, 0.68056935, 0.650217, 0.5829565]","[0.7595998, 0.7571845, 0.5400534, 0.808226]"


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel
import torch

# Load CSV file
df = pd.read_csv('/content/drive/My Drive/Dataset/ml_dataset.csv', skiprows=0)

# Process only the first 5 questions
results = []
question_count = 0

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

for index, row in df.iterrows():
    if question_count >= 5:
        break  # Exit the loop if we have processed 5 questions

    model_answer = row[6]  # Assuming the model answer is in the last column
    sample_answers = row[2:6].tolist()  # Assuming sample answers are in columns 3 to 6

    # Word2Vec for Cosine Similarity
    # Train Word2Vec model
    text_data = [model_answer] + sample_answers
    tokenized_data = [str(text).split() for text in text_data]
    word2vec_model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

    # Calculate cosine similarity between each sample answer and model answer using Word2Vec
    similarity_scores_word2vec = []
    for sample_answer in sample_answers:
        model_answer_vec = [word2vec_model.wv[word] for word in str(model_answer).split() if word in word2vec_model.wv]
        sample_answer_vec = [word2vec_model.wv[word] for word in str(sample_answer).split() if word in word2vec_model.wv]
        if len(sample_answer_vec) > 0 and len(model_answer_vec) > 0:
            similarity_score = cosine_similarity(sample_answer_vec, model_answer_vec).mean()
        else:
            similarity_score = 0.0
        similarity_scores_word2vec.append(round((similarity_score + 1) * 50, 4))  # Scaling to 0-100 range and rounding to 4 decimal places

    # TF-IDF for Cosine Similarity
    # Calculate TF-IDF vectors
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([model_answer] + sample_answers)

    # Calculate cosine similarity between each sample answer and model answer using TF-IDF
    similarity_scores_tfidf = []
    for i in range(1, tfidf_matrix.shape[0]):
        similarity_score = cosine_similarity(tfidf_matrix[0], tfidf_matrix[i]).mean()
        similarity_scores_tfidf.append(round((similarity_score + 1) * 50, 4))  # Scaling to 0-100 range and rounding to 4 decimal places

    # BERT embeddings
    # Tokenize model answer and sample answers
    model_answer_tokens = tokenizer.tokenize(model_answer)
    sample_answers_tokens = [tokenizer.tokenize(sample_answer) for sample_answer in sample_answers]

    # Convert tokens to IDs
    model_answer_ids = tokenizer.convert_tokens_to_ids(model_answer_tokens)
    sample_answers_ids = [tokenizer.convert_tokens_to_ids(sample_answer_tokens) for sample_answer_tokens in sample_answers_tokens]

    # Convert IDs to tensors
    model_answer_tensor = torch.tensor([model_answer_ids])
    sample_answers_tensors = [torch.tensor([sample_answer_ids]) for sample_answer_ids in sample_answers_ids]

    # Get BERT embeddings for model answer
    with torch.no_grad():
        model_answer_embedding = bert_model(model_answer_tensor)[0][:, 0, :].numpy()

    # Calculate cosine similarity between each sample answer and model answer using BERT embeddings
    similarity_scores_bert = []
    for sample_answer_tensor in sample_answers_tensors:
        with torch.no_grad():
            sample_answer_embedding = bert_model(sample_answer_tensor)[0][:, 0, :].numpy()
        similarity_score = cosine_similarity(sample_answer_embedding, model_answer_embedding).mean()
        similarity_scores_bert.append(round((similarity_score + 1) * 50, 4))  # Scaling to 0-100 range and rounding to 4 decimal places

    # SBERT for Similarity
    sbert_model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

    # Encode model answer
    model_answer_embedding_sbert = sbert_model.encode(str(model_answer))

    # Calculate cosine similarity between each sample answer and model answer using SBERT
    similarity_scores_sbert = []
    for sample_answer in sample_answers:
        sample_answer_embedding_sbert = sbert_model.encode(str(sample_answer))
        similarity_score = cosine_similarity([model_answer_embedding_sbert], [sample_answer_embedding_sbert])[0][0]
        similarity_scores_sbert.append(round((similarity_score + 1) * 50, 4))  # Scaling to 0-100 range and rounding to 4 decimal places

    # Append results for this question to the list
    results.append({
        'Similarity (Word2Vec)': similarity_scores_word2vec,
        'Similarity (TF-IDF)': similarity_scores_tfidf,
        'Similarity (BERT)': similarity_scores_bert,
        'Similarity (SBERT)': similarity_scores_sbert
    })

    question_count += 1

# Display results
results_df = pd.DataFrame(results)
results_df


,Similarity (Word2Vec),Similarity (TF-IDF),Similarity (BERT),Similarity (SBERT)
0,"[51.368, 51.0683, 51.397, 51.3761]","[80.7081, 73.4594, 79.7933, 79.2312]","[90.5467, 93.8066, 95.3832, 90.2146]","[88.6068, 93.2192, 93.7548, 94.2395]"
1,"[51.0826, 50.8101, 50.6588, 50.7992]","[65.989, 69.6609, 66.2574, 66.511]","[85.9045, 88.1656, 91.2999, 87.7298]","[84.9079, 86.9595, 69.0477, 86.8297]"
2,"[51.6945, 51.6745, 51.5037, 51.5086]","[76.191, 78.5439, 72.3558, 77.1408]","[91.863, 78.3384, 82.8011, 88.4677]","[93.0091, 88.4907, 91.0221, 90.2745]"
3,"[51.0407, 51.2193, 50.9727, 50.8853]","[62.471, 68.1281, 64.7534, 67.9034]","[83.4338, 89.6362, 86.675, 89.536]","[82.4991, 87.7481, 88.3375, 90.6393]"
4,"[51.5945, 51.6832, 51.4177, 51.8768]","[70.4577, 70.2, 66.141, 74.2168]","[90.5019, 84.0285, 82.5108, 79.1478]","[87.98, 87.8592, 77.0027, 90.4113]"


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch

# Load CSV file
df = pd.read_csv('/content/drive/My Drive/Dataset/ml_dataset.csv', skiprows=0)

# Process only the first 5 questions
results = []
question_count = 0

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

for index, row in df.iterrows():
    if question_count >= 5:
        break  # Exit the loop if we have processed 5 questions

    model_answer = row[6]  # Assuming the model answer is in the last column
    sample_answers = row[2:6].tolist()  # Assuming sample answers are in columns 3 to 6

    # BERT embeddings
    # Tokenize model answer and sample answers
    model_answer_tokens = tokenizer.tokenize(model_answer)
    sample_answers_tokens = [tokenizer.tokenize(sample_answer) for sample_answer in sample_answers]

    # Convert tokens to IDs
    model_answer_ids = tokenizer.convert_tokens_to_ids(model_answer_tokens)
    sample_answers_ids = [tokenizer.convert_tokens_to_ids(sample_answer_tokens) for sample_answer_tokens in sample_answers_tokens]

    # Convert IDs to tensors
    model_answer_tensor = torch.tensor([model_answer_ids])
    sample_answers_tensors = [torch.tensor([sample_answer_ids]) for sample_answer_ids in sample_answers_ids]

    # Get BERT embeddings for model answer
    with torch.no_grad():
        model_answer_embedding = bert_model(model_answer_tensor)[0][:, 0, :].numpy()

    # Calculate cosine similarity between each sample answer and model answer using BERT embeddings
    similarity_scores_bert = []
    for sample_answer_tensor in sample_answers_tensors:
        with torch.no_grad():
            sample_answer_embedding = bert_model(sample_answer_tensor)[0][:, 0, :].numpy()
        similarity_score = cosine_similarity(sample_answer_embedding, model_answer_embedding).mean()
        similarity_scores_bert.append(round((similarity_score + 1) * 50, 4))  # Scaling to 0-100 range and rounding to 4 decimal places

    # SBERT for Similarity
    sbert_model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

    # Encode model answer
    model_answer_embedding_sbert = sbert_model.encode(str(model_answer))

    # Calculate cosine similarity between each sample answer and model answer using SBERT
    similarity_scores_sbert = []
    for sample_answer in sample_answers:
        sample_answer_embedding_sbert = sbert_model.encode(str(sample_answer))
        similarity_score = cosine_similarity([model_answer_embedding_sbert], [sample_answer_embedding_sbert])[0][0]
        similarity_scores_sbert.append(round((similarity_score + 1) * 50, 4))  # Scaling to 0-100 range and rounding to 4 decimal places

    # Append results for this question to the list
    results.append({
        'Similarity (BERT)': similarity_scores_bert,
        'Similarity (SBERT)': similarity_scores_sbert
    })

    question_count += 1

# Display results
results_df = pd.DataFrame(results)
results_df


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

,Similarity (BERT),Similarity (SBERT)
0,"[90.5467, 93.8066, 95.3832, 90.2146]","[88.6068, 93.2192, 93.7548, 94.2395]"
1,"[85.9045, 88.1656, 91.2999, 87.7298]","[84.9079, 86.9595, 69.0477, 86.8297]"
2,"[91.863, 78.3384, 82.8011, 88.4676]","[93.0091, 88.4907, 91.0221, 90.2745]"
3,"[83.4338, 89.6362, 86.6749, 89.536]","[82.4991, 87.7481, 88.3375, 90.6393]"
4,"[90.5019, 84.0285, 82.5108, 79.1478]","[87.98, 87.8592, 77.0027, 90.4113]"
